# Custom Researcher Workflow
This notebook demonstrates how to create a custom workflow using the `AgentWorkflow` class.

This example extract contents from 3 urls, extract insights from the contents, review the insights, and provide a detailed report based on the insights.
- "https://www.anthropic.com/engineering/building-effective-agents",
- "https://www.ibm.com/think/topics/agentic-workflows",
- "https://aws.amazon.com/blogs/hpc/building-an-ai-simulation-assistant-with-agentic-workflows/"

# Imports

In [1]:
import asyncio
from dotenv import load_dotenv
from typing import Any, Dict
import requests
from bs4 import BeautifulSoup

from saw.core.model_interface import amodel_call
from saw.workflow import AgentWorkflow
from saw.workflows.utils import aapply_functions

In [2]:
# Load environment variables
load_dotenv("../docker/.env")

True

# Define Custom Researcher Function
Define a custom function that will be used by the `AgentWorkflow` to research content.


In [3]:
async def add_title(content: str, title: str) -> str:
    """Add a title to the top of the content.

    Args:
        content (str): The content to which the title will be added.
        title (str): The title to add.

    Returns:
        str: The content with the title added at the top.
    """
    return f"{title}\n\n{content}"


async def fetch_content(url: str) -> str:
    """Fetch content from a given URL.

    Args:
        url (str): The URL to fetch content from.

    Returns:
        str: The content of the URL
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

In [4]:
async def custom_researcher(researcher: Dict[str, Any], principal_researcher: Dict[str, Any], tech_lead: Dict[str, Any]) -> tuple:
    """Custom Researcher Function.

    Args:
        researcher (Dict[str, Any]): Dictionary containing details for the researcher role.
        principal_researcher (Dict[str, Any]): Dictionary containing details for the principal researcher role.
        tech_lead (Dict[str, Any]): Dictionary containing details for the tech lead role.

    Returns:
        tuple: The results of the custom researcher functions.
    """
    # Using the aapply_functions function to apply the fetch_content function
    contents = await asyncio.gather(
        *[aapply_functions(prompt=url, functions=researcher["functions"])
          for url in researcher["params"]['urls']]
    )

    print(f"Contents ({type(contents)}): {contents}")

    insights = []
    for content in contents:
        print(f"Extracting insights from: {content}")
        insight = await amodel_call(
            prompt=f"{researcher['prompt']}\n{content}",
            system_prompt=researcher['system_prompt'],
            provider=researcher['provider'],
            model=researcher['model']
        )
        insights.append(insight)

    print(f"Insights: {insights}")

    principal_researcher_approval = await amodel_call(
        prompt=f"{principal_researcher['prompt']}\n{insights}",
        system_prompt=principal_researcher['system_prompt'],
        provider=principal_researcher['provider'],
        model=principal_researcher['model']
    )

    if (principal_researcher_approval and
            "more details" in principal_researcher_approval.lower()):
        insights = []
        for content in contents:
            insight = await amodel_call(
                prompt=f"{researcher['prompt']}\n{content}",
                system_prompt=researcher['system_prompt'],
                provider=researcher['provider'],
                model=researcher['model']
            )
            insights.append(insight)

    # Directly using the add_title function to add a title to the pr_report
    pr_report = await principal_researcher['functions'][0](
        title=principal_researcher['params']['title'],
        content=principal_researcher_approval
    )
    print(f"Principal Researcher Approval: {pr_report}")

    tech_lead_report = await amodel_call(
        prompt=f"{tech_lead['prompt']}\n{insights}",
        system_prompt=tech_lead['system_prompt'],
        provider=tech_lead['provider'],
        model=tech_lead['model']
    )

    # Directly using the add_title function to add a title to the pr_report
    tl_report = await tech_lead['functions'][0](
        title=tech_lead['params']['title'],
        content=tech_lead_report
    )
    print(f"Tech Lead Report: {tl_report}")

    return insights, pr_report, tl_report

# Initialize Agent Workflow

In [5]:
# Initialize the `AgentWorkflow` with the custom researcher function.
agent = AgentWorkflow(operation="custom", custom_workflow=custom_researcher)

In [6]:
# Define dictionaries for each role
researcher = {
    "prompt": "Research on agents and agentic workflows.",
    "system_prompt": "You are a researcher.",
    "functions": [fetch_content],
    "provider": "google",
    "model": "gemini-2.0-flash",
    "params": {
        "urls": [
            "https://www.anthropic.com/engineering/building-effective-agents",
            "https://www.ibm.com/think/topics/agentic-workflows",
            "https://aws.amazon.com/blogs/hpc/building-an-ai-simulation-assistant-with-agentic-workflows/"
        ]
    }
}

principal_researcher = {
    "prompt": "Analyze the key findings from the research on agents and agentic workflows.",
    "system_prompt": "You are a principal researcher.",
    "functions": [add_title],
    "model": "deepseek-r1:1.5b",
    "provider": "ollama",
    "params": {
        "title": "Principal Researcher Report"
    }
}

tech_lead = {
    "prompt": "Write a detailed report based on the key findings from the research on "
              "agents and agentic workflows.",
    "system_prompt": "You are a tech lead.",
    "functions": [add_title],
    "model": "gemini-2.0-flash",
    "provider": "google",
    "params": {
        "title": "Tech Lead Report"
    }
}

In [7]:
# Execute the workflow
insights, principal_researcher_report, tech_lead_report = await agent.execute(
    researcher=researcher,
    principal_researcher=principal_researcher,
    tech_lead=tech_lead,
    async_mode=True
)

Function Arguments: {'researcher': {'prompt': 'Research on agents and agentic workflows.', 'system_prompt': 'You are a researcher.', 'functions': [<function fetch_content at 0x7673e1f13ec0>], 'provider': 'google', 'model': 'gemini-2.0-flash', 'params': {'urls': ['https://www.anthropic.com/engineering/building-effective-agents', 'https://www.ibm.com/think/topics/agentic-workflows', 'https://aws.amazon.com/blogs/hpc/building-an-ai-simulation-assistant-with-agentic-workflows/']}}, 'principal_researcher': {'prompt': 'Analyze the key findings from the research on agents and agentic workflows.', 'system_prompt': 'You are a principal researcher.', 'functions': [<function add_title at 0x7673e1f13e20>], 'model': 'deepseek-r1:1.5b', 'provider': 'ollama', 'params': {'title': 'Principal Researcher Report'}}, 'tech_lead': {'prompt': 'Write a detailed report based on the key findings from the research on agents and agentic workflows.', 'system_prompt': 'You are a tech lead.', 'functions': [<function

In [8]:
print(principal_researcher_report)

Principal Researcher Report

<think>
Okay, so I'm trying to understand how to apply AI, specifically LLMs like Anthropic's Claude, to simulation workflows. The blog post by the principal is a bit dense, but let me try to break it down.

First, I see that simulations are being democratized through LLMs. They're using cloud-based architectures with AWS services, which makes sense because it offers scalability and efficiency. But how exactly do these LLMs act as agents? The blog post talks about the "Tool Use" pattern, where the LLM acts as a tool to launch simulations or analyze results. That seems logical because LLMs can understand natural language requests and break them down into actionable steps.

I'm a bit confused about how structured chat agents fit in here. They mention using bedrock for multi-input tools, which means each input field is handled by its own tool. This makes sense as it adds modularity to the system. But I wonder if they need to integrate with LLMs or if the LLM c

In [9]:
print(tech_lead_report)

Tech Lead Report

Okay, based on the research reviewed, here's a detailed report on agents and agentic workflows, outlining key findings, challenges, and future directions:

## Report: Agents and Agentic Workflows: A Deep Dive

**Executive Summary:**

This report analyzes recent research on agents and agentic workflows, emphasizing the transformative potential of AI-driven automation. Agentic workflows leverage autonomous AI agents, often powered by Large Language Models (LLMs), to dynamically plan, execute, and adapt complex tasks with minimal human intervention. The report highlights key components, challenges, and emerging trends in the field, ultimately suggesting avenues for further investigation and development.  The current research strongly supports that LLMs are effective as orchestrators.

**1. Introduction:**

The paradigm of automating complex tasks is shifting from rigid, rule-based systems to more flexible and intelligent agentic workflows. An *agent* is defined as an ent